In [ ]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from whi_utils import * 

In [ ]:
df_combined = pd.read_csv("/data/whi/data/main_study/processed/whi_combined_nocolinear.csv")  # combined dataset  
df_combined["T"] = df_combined["T"] / df_combined["T"].max()

A = 0
df_combined = df_combined.loc[df_combined.A == A].copy()

df_obs = df_combined.query("S==1").drop(["S", "Delta", "const"], axis=1).reset_index(drop=True)
df_comp = df_combined.query("S==0").reset_index(drop=True)

nrct = 1000
bfr = 0.4

sb_arr = np.random.choice(df_comp.loc[df_comp.Delta==1].index, int(nrct * bfr))
df_half1_rct = df_comp.loc[sb_arr].copy().reset_index(drop=True)
df_comp.drop(sb_arr, axis=0, inplace=True)
df_comp.reset_index(drop=True)

nb_arr = np.random.choice(df_comp.index, int(nrct * (1 - bfr)))
df_half2_rct = df_comp.loc[nb_arr].copy().reset_index(drop=True)
df_comp.drop(nb_arr, axis=0, inplace=True)
df_comp.reset_index(drop=True)

df_rct = pd.concat([df_half1_rct, df_half2_rct], ignore_index=True, axis=0)

df_rct = df_rct.drop(["S", "Delta", "const"], axis=1).reset_index(drop=True)
df_tar = df_comp.drop(["S", "Delta", "const"], axis=1).reset_index(drop=True)

In [ ]:
regressor_list = [*df_obs.columns[2:]]

f_a_X = fit_obs_outcome_fn(df_obs, regressors=regressor_list, target="T")
g_a_X = fit_trial_outcome_fn(df_rct, regressors=regressor_list, target="T")

df_rct['fa(X)'] = f_a_X.predict(np.array(df_rct[regressor_list]))
df_rct['Z'] = df_rct['fa(X)'] - df_rct['T']

df_tar['fa(X)'] = f_a_X.predict(np.array(df_tar[regressor_list]))
df_tar['ga(X)'] = g_a_X.predict(np.array(df_tar[regressor_list]))

b_a_X = fit_trial_bias_fn(df_rct, regressors=regressor_list, target="Z")
df_tar['ba(X)'] = b_a_X.predict(np.array(df_tar[regressor_list]))

h_a_X = fit_trial_outcome_fn(df_rct, regressors=regressor_list + ["fa(X)"], target="T")
df_tar['ha(X)'] = h_a_X.predict(np.array(df_tar[regressor_list + ["fa(X)"]]))


In [ ]:
gt = df_tar['T'].mean()
bsl1 = df_tar['fa(X)'].mean()
bsl2 = df_tar['ga(X)'].mean()
abc = df_tar['fa(X)'].mean() - df_tar['ba(X)'].mean()
aom = df_tar['ha(X)'].mean()